# 5-1 Assignment: Cartpole Problem
___
<div class="alert alert-block alert-success" style="color:black;">
<b>To Begin:</b> Run all code blocks and observe the output. Once you have reviewed the sample output. Use the <b>LastName_FirstName_Assignment2.ipynb</b> file to complete your assignment.
</div>

<div class="alert alert-block alert-warning" style="color:black;">
    <b>Note:</b> For compatability purposes, libraries have been updated from those used in the required readings to match to current versions; hence some of the package invocations may differ slightly from the book. The affected lines of code have comments added to the right as applicable, with the old code commented out above for reference.
</div>

<div class="alert alert-block alert-danger" style="color:black;">
<b>GPU/CUDA/Memory Warnings/Errors:</b> You may receive some errors referencing that GPUs will not be used, CUDA could not be found, or free system memory allocation errors. These and a few others, are standard errors that can be ignored here as they are environment based.<br><br>
<b>Example messages:</b>
    <ul>
        <li>Could not find cuda drivers on your machine, GPU will not be used.</li>
        <li>Please check linkage and avoid linking the same target more than once.</li>
        <li>E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)</li>
        <li>Allocation of ######## exceeds 10% of free system memory</li>
    </ul>
</div>

---

### Installing Required Packages
This is to install necessary components to run the assignment

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.1.2 -> 26.0
[notice] To update, run: pip install --upgrade pip


<div class="alert alert-block alert-warning" style="color:black;">
    <b>Note:</b> If the assignment takes too long, you may want to download the code and run it locally to take advantage of your GPU.
</div>

In [2]:
# Forces TensorFlow to use CPU only
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Package Imports
import numpy as np
import tensorflow as tf
import gymnasium as gym
from collections import deque
import random

# Environment setup and Variables
try:
    env = gym.make("CartPole-v1", render_mode=None)
except Exception:
    print('Failed to initialize environment! Make sure that gymnasium was installed correctly!')
    sys.exit(1)

state_shape = env.observation_space.shape[0]
action_size = env.action_space.n

# Hyperparameters
GAMMA = 0.99
EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.990
LEARNING_RATE = 0.001
BATCH_SIZE = 64
TRAIN_START = 1000
MEMORY_SIZE = 2000
EPISODES = 300

# Counters during training
train_freq = 4
step_count = 0
target_update_freq = 10
# If an error occurs below, it is because the environment is looking for a GPU.

2026-02-05 20:38:32.583573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-05 20:38:32.605821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-05 20:38:32.612403: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
memory = deque(maxlen=MEMORY_SIZE)

# DQN Builder
def build_dqn_model():
    try:
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(state_shape,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='mse')
        return model
    except Exception:
        print('Error occurred while building the DQN model!')
        sys.exit(1)

In [4]:
# Calling the build_dqn_model method
model = build_dqn_model()
target_model = build_dqn_model()
target_model.set_weights(model.get_weights())

2026-02-05 20:38:51.327872: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


<div class="alert alert-block alert-warning" style="color:black;">
    <b>Note:</b> If an error displays, it is simply trying to connect to the GPU.
</div>


In [5]:
# Random Action selection for exploration
def get_action(state, EXPLORATION_MAX):
    if np.random.rand() <= EXPLORATION_MAX:
        return random.randrange(action_size)
    q_values = model.predict(np.array([state]), verbose=0)  # quiet predict
    return np.argmax(q_values[0])

# def get_action(state, EXPLORATION_MAX):
#     try:
#         if np.random.rand() <= EXPLORATION_MAX:
#             return random.randrange(action_size)
#         q_values = model.predict(np.array([state]), verbose=0) #Remove verbose=0 to see every prediction display
#         return np.argmax(q_values[0])
#     except Exception:
#             print("Error during get_action method!")
#             sys.exit(1)

In [6]:
# Experience Replay
def experience_replay():
    if len(memory) < TRAIN_START:
        return

    batch = random.sample(memory, BATCH_SIZE)
    states = np.zeros((BATCH_SIZE, state_shape))
    next_states = np.zeros((BATCH_SIZE, state_shape))
    actions, rewards, completions = [], [], []

    try:
        for i, (state, action, reward, state_next, terminal) in enumerate(batch):
            states[i] = state
            next_states[i] = state_next
            actions.append(action)
            rewards.append(reward)
            completions.append(terminal)

        target_q = model.predict(states, verbose=0)           #Remove verbose=0 to see every prediction display
        next_q = target_model.predict(next_states, verbose=0) #Remove verbose=0 to see every prediction display

        for i in range(BATCH_SIZE):
            if completions[i]:
                target_q[i][actions[i]] = rewards[i]
            else:
                target_q[i][actions[i]] = rewards[i] + GAMMA * np.max(next_q[i])

        model.fit(states, target_q, batch_size=BATCH_SIZE, verbose=0)
    except Exception:
        print("Error during replay")
        sys.exit(1)

In [7]:
# Quiet training function
def train_cartpole(GAMMA_val=0.99, EXPLORATION_MAX_val=1.0, 
                   EXPLORATION_MIN_val=0.01, EXPLORATION_DECAY_val=0.990, 
                   LEARNING_RATE_val=0.001, EPISODES_val=300):
    
    # Resets memory, counters, and models between experiments
    global memory, step_count, model, target_model
    memory = deque(maxlen=MEMORY_SIZE)
    step_count = 0
    
    # Overrides hyperparameters for this run
    GAMMA_local = GAMMA_val
    EXPLORATION_MAX_local = EXPLORATION_MAX_val
    EXPLORATION_MIN_local = EXPLORATION_MIN_val
    EXPLORATION_DECAY_local = EXPLORATION_DECAY_val
    LEARNING_RATE_local = LEARNING_RATE_val
    
    # Rebuilds models
    model = build_dqn_model()
    target_model = build_dqn_model()
    target_model.set_weights(model.get_weights())
    
    # Episode rewards for this run
    episode_rewards_local = []
    
    for e in range(EPISODES_val):
        state, _ = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            if np.random.rand() <= EXPLORATION_MAX_local:
                action = random.randrange(action_size)
            else:
                q_values = model.predict(np.array([state]), verbose=0)
                action = np.argmax(q_values[0])
            
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            reward = reward if not done else -10
            
            memory.append((state, action, reward, next_state, done))
            state = next_state
            total_reward += reward
            
            step_count += 1
            if step_count % train_freq == 0:
                experience_replay()
        
        # Updates exploration
        if EXPLORATION_MAX_local > EXPLORATION_MIN_local:
            EXPLORATION_MAX_local *= EXPLORATION_DECAY_local
            EXPLORATION_MAX_local = max(EXPLORATION_MIN_local, EXPLORATION_MAX_local)
        
        # Updates target model
        if e % target_update_freq == 0:
            target_model.set_weights(model.get_weights())
        
        episode_rewards_local.append(total_reward)
        
        # Prints every 10 episodes
        if e % 10 == 0:
            avg_reward = np.mean(episode_rewards_local[-10:])
            print(f"Episode {e+1}: Avg Reward = {avg_reward:.2f}, Exploration = {EXPLORATION_MAX_local:.3f}")
    
    print("Training complete for this experiment!")
    return episode_rewards_local

In [8]:
# Experiment 1: Default Params
rewards_exp1 = train_cartpole()

Episode 1: Avg Reward = 15.00, Exploration = 0.990
Episode 11: Avg Reward = 10.10, Exploration = 0.895
Episode 21: Avg Reward = 7.30, Exploration = 0.810
Episode 31: Avg Reward = 4.70, Exploration = 0.732
Episode 41: Avg Reward = 5.00, Exploration = 0.662
Episode 51: Avg Reward = 7.20, Exploration = 0.599
Episode 61: Avg Reward = 1.70, Exploration = 0.542
Episode 71: Avg Reward = 1.40, Exploration = 0.490
Episode 81: Avg Reward = 0.40, Exploration = 0.443
Episode 91: Avg Reward = 0.30, Exploration = 0.401
Episode 101: Avg Reward = 1.10, Exploration = 0.362
Episode 111: Avg Reward = 0.70, Exploration = 0.328
Episode 121: Avg Reward = 1.00, Exploration = 0.296
Episode 131: Avg Reward = 1.10, Exploration = 0.268
Episode 141: Avg Reward = 25.90, Exploration = 0.242
Episode 151: Avg Reward = 71.20, Exploration = 0.219
Episode 161: Avg Reward = 57.30, Exploration = 0.198
Episode 171: Avg Reward = 68.70, Exploration = 0.179
Episode 181: Avg Reward = 88.50, Exploration = 0.162
Episode 191: Avg

KeyboardInterrupt: 

In [9]:
# Experiment 2: Lower discount factor, slower exploration decay
rewards_exp2 = train_cartpole(GAMMA_val=0.8, EXPLORATION_DECAY_val=0.995)

Episode 1: Avg Reward = 14.00, Exploration = 0.995
Episode 11: Avg Reward = 13.60, Exploration = 0.946
Episode 21: Avg Reward = 16.50, Exploration = 0.900
Episode 31: Avg Reward = 17.00, Exploration = 0.856
Episode 41: Avg Reward = 15.50, Exploration = 0.814
Episode 51: Avg Reward = 11.50, Exploration = 0.774
Episode 61: Avg Reward = 13.10, Exploration = 0.737
Episode 71: Avg Reward = 14.10, Exploration = 0.701
Episode 81: Avg Reward = 23.90, Exploration = 0.666
Episode 91: Avg Reward = 40.50, Exploration = 0.634


KeyboardInterrupt: 

In [10]:
# Experiment 3: Higher learning rate
rewards_exp3 = train_cartpole(LEARNING_RATE_val=0.005)

Episode 1: Avg Reward = 13.00, Exploration = 0.990
Episode 11: Avg Reward = 8.90, Exploration = 0.895
Episode 21: Avg Reward = 5.80, Exploration = 0.810
Episode 31: Avg Reward = 12.10, Exploration = 0.732
Episode 41: Avg Reward = 9.40, Exploration = 0.662
Episode 51: Avg Reward = 7.50, Exploration = 0.599
Episode 61: Avg Reward = 5.50, Exploration = 0.542
Episode 71: Avg Reward = 4.10, Exploration = 0.490
Episode 81: Avg Reward = 11.20, Exploration = 0.443
Episode 91: Avg Reward = 38.90, Exploration = 0.401
Episode 101: Avg Reward = 37.60, Exploration = 0.362
Episode 111: Avg Reward = 64.10, Exploration = 0.328
Episode 121: Avg Reward = 153.20, Exploration = 0.296


KeyboardInterrupt: 

<div class="alert alert-block alert-warning" style="color:black;">
    <b>Note:</b><br>
    If an error displays, it is attempting to connect to the GPU.<br>
    It will not connect and run on CPU. <br>
    Code will take some time to run, which is commonplace for real life models.<br>
    <b><i>-- Make sure your computer does not go to sleep, and take a well-deserved break! --</b></i>
</div>


# Analysis of Hyperparameter Experiments

Note: The experiments were run in a CPU-only Codio environment, so training was naturally slower than on a GPU. Some long-running sessions triggered kernel interruptions and TensorFlow warnings related to GPU availability were observed, but all results shown reflect the agent’s learning under these conditions.

## Experiment 1: Default Parameters
With the default setup, the CartPole agent starts off performing poorly, which isn’t surprising given the high exploration rate (ε ≈ 0.99); early episodes are mostly random actions, thus resulting in low rewards (FreeCodeCamp, 2018). As training progresses and the agent begins to rely more on what it has learned, the average reward sharply increases after episode 140 (Gulli & Pal, 2017). This behavior demonstrates the balance of exploration and exploitation inherent in ε-greedy strategies as the agent initially experiments broadly before gradually focusing on the most effective actions (FreeCodeCamp, 2018; Gulli & Pal, 2017). In the CartPole environment, the agent’s goal is to keep the pole upright for as long as possible. The state space includes four key values: cart position, cart velocity, pole angle, and pole angular velocity. The agent can take one of two discrete actions at each time step: move the cart left or move it right. Experience replay helps stabilize learning by letting the agent train on a diverse set of past experiences rather than only the most recent moves, which explains the dramatic improvements that we see later in training (FreeCodeCamp, 2018). Experience tuples of the form (state, action, reward, next state) are stored in a replay memory and sampled randomly during training to reduce correlation between consecutive experiences, which leads to improved convergence stability (Gulli & Pal, 2017).

## Experiment 2: Lower Discount Factor/Slower Exploration Decay
When the discount factor was reduced to γ = 0.8 and the decay of exploration slowed down, the agent’s learning curve became smoother (FreeCodeCamp, 2018). By focusing more on immediate rewards as opposed to heavily weighting future rewards, the agent’s progress is steadier but the maximum reward achieved is slightly lower within the same number of episodes. By decaying exploration more gradually, the agent then continues experimenting with different actions for longer, which then reduces the risk of prematurely converging on suboptimal strategies (SerengetiTech, 2019). This demonstrates how careful adjustment of the discount factor and exploration schedule is able to shape both the speed of learning and the consistency of outcomes. The discount factor γ influences how much future rewards are valued relative to immediate rewards, which helps guide the agent to consider long-term consequences when selecting actions (Gulli & Pal, 2017).

## Experiment 3: Higher Learning Rate
Increasing the learning rate to α = 0.005 made the agent adapt more quickly to rewards from the environment (FreeCodeCamp, 2018). Early in training, this caused more variability in performance and fluctuations in average reward, but the agent ended up reaching higher rewards faster than with the default rate over time. This experiment demonstrates the balance between learning speed and stability in Q-learning: a higher learning rate can accelerate learning, yet if set too high, it could prevent the network from properly converging (Gulli & Pal, 2017). These results emphasize the importance of carefully tuning the learning rate to achieve efficient learning while also minimizing instability. For this CartPole problem, a deep Q-network (DQN) is used, where a neural network approximates the Q-function. The network takes the state vector as input, passes it through hidden layers (dense layers with ReLU activation), and outputs Q-values for each possible action. By generalizing across the continuous state space, the neural network makes Q-learning feasible for large or continuous environments (Gulli & Pal, 2017; FreeCodeCamp, 2018). Adjusting the learning rate directly affects how quickly the network updates Q-value estimates, impacting both learning speed and convergence stability (Gulli & Pal, 2017).

### References
FreeCodeCamp. (2018, September 3). *An introduction to Q-learning: Reinforcement learning.* Medium. https://medium.com/free-code-camp/an-introduction-to-q-learning-reinforcement-learning-14ac0b4493cc

Gulli, A., & Pal, S. (2017). *Deep learning with Keras: Implementing deep learning models and neural networks with the power of Python* (Chapter 8). Packt Publishing.

SerengetiTech. (2019, June 18). *Using Q-learning for pathfinding.* https://serengetitech.com/tech/using-q-learning-for-pathfinding/